<a href="https://colab.research.google.com/github/Onejledonlala/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").getOrCreate()

In [3]:
# Import struct fields that we can use
from pyspark.sql.types import StructField, StringType, IntegerType, StructType, DateType

In [7]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()
df.count()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

904765

In [12]:
# Create the vine_table DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase']).drop_duplicates().dropna()
vine_df.show()
vine_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2WYOW5FJN2E4I|          5|            0|          0|   N|                N|
| RVP3NRG5JTZR7|          5|            0|          0|   N|                Y|
|R1SH132MVL0CLB|          4|            0|          0|   N|                N|
|R3L85PUNW3V73X|          4|            1|          1|   N|                N|
|R28X8S4T59RYMN|          5|            0|          0|   N|                Y|
|R2L6TXJGKC8UGO|          4|            1|          1|   N|                Y|
|R10ZZUFOF1GPCA|          1|            1|          1|   N|                Y|
|R1FPG5OGUT1CAY|          5|            1|          1|   N|                Y|
|R34PXE0JU9CPOC|          3|            8|          8|   N|                Y|
| R2HA7CHRRTUWH|          5|            1|          1|   N|     

904763

In [38]:
# filter by total_votes being 20 or more
vine_votes_df = vine_df.filter('total_votes>=20')
vine_votes_df.show()
vine_votes_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1TB1H7N9N35M3|          5|           25|         31|   N|                Y|
| RKYLHZL7EPELX|          4|           19|         25|   Y|                N|
|R2V2O0OBZVHZV4|          5|           43|         44|   N|                Y|
|R3RJZ6WEJWEVTW|          4|           22|         24|   N|                N|
| RA229OUGA4JAF|          5|           26|         28|   N|                Y|
| RDY3FINBWZEDG|          5|           26|         28|   N|                Y|
|R2BOAXILS5AV6T|          5|           46|         48|   N|                N|
| RC8JVI01ETSO5|          1|            0|         28|   N|                Y|
|R3J7GKYADD0KCL|          4|           21|         22|   N|                Y|
| RB1H5YH6HOIYP|          5|           22|         24|   N|     

16520

In [50]:
# filter new dataframe where the number of helpful_votes divided by total_votes is equal to or greater than 50%
vine_helpful_df = vine_votes_df.filter('(helpful_votes/total_votes) >= 0.5')
vine_helpful_df.show()
vine_helpful_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1TB1H7N9N35M3|          5|           25|         31|   N|                Y|
| RKYLHZL7EPELX|          4|           19|         25|   Y|                N|
|R2V2O0OBZVHZV4|          5|           43|         44|   N|                Y|
|R3RJZ6WEJWEVTW|          4|           22|         24|   N|                N|
| RA229OUGA4JAF|          5|           26|         28|   N|                Y|
| RDY3FINBWZEDG|          5|           26|         28|   N|                Y|
|R2BOAXILS5AV6T|          5|           46|         48|   N|                N|
|R3J7GKYADD0KCL|          4|           21|         22|   N|                Y|
| RB1H5YH6HOIYP|          5|           22|         24|   N|                N|
|R16EJZJVR8C4DN|          5|           13|         20|   N|     

14537

In [51]:
# filter newer dataframe where a review was written as part of the Vine program (paid)
vine_paid_df = vine_helpful_df.filter(vine_helpful_df["vine"] == "Y")
vine_paid_df.show()
vine_paid_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RKYLHZL7EPELX|          4|           19|         25|   Y|                N|
|R3QQG4HUB214TM|          5|          111|        128|   Y|                N|
|R19EFYNN3W8Q07|          5|           26|         32|   Y|                N|
|R2SW4NXNO7HZJ5|          4|           28|         33|   Y|                N|
|R2XWB7UZBANOS3|          4|           18|         20|   Y|                N|
| RYW05F1MUEF01|          5|           87|        102|   Y|                N|
| RARYFB85E8YSK|          5|           25|         31|   Y|                N|
|R2VRYZV3N49TJ0|          4|           29|         33|   Y|                N|
|R22CYBPLKWDK6F|          5|           47|         54|   Y|                N|
| RP24DFOZY82Z9|          5|           24|         26|   Y|     

60

In [52]:
# filter newer dataframe where a review was not written as part of the Vine program (unpaid)
vine_unpaid_df = vine_helpful_df.filter(vine_helpful_df["vine"] == "N")
vine_unpaid_df.show()
vine_unpaid_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1TB1H7N9N35M3|          5|           25|         31|   N|                Y|
|R2V2O0OBZVHZV4|          5|           43|         44|   N|                Y|
|R3RJZ6WEJWEVTW|          4|           22|         24|   N|                N|
| RA229OUGA4JAF|          5|           26|         28|   N|                Y|
| RDY3FINBWZEDG|          5|           26|         28|   N|                Y|
|R2BOAXILS5AV6T|          5|           46|         48|   N|                N|
|R3J7GKYADD0KCL|          4|           21|         22|   N|                Y|
| RB1H5YH6HOIYP|          5|           22|         24|   N|                N|
|R16EJZJVR8C4DN|          5|           13|         20|   N|                Y|
|R3GVZIEF9SDZKG|          4|           24|         25|   N|     

14477

In [53]:
# Review counts
print("The total number of reviews are " + str(vine_df.count()))

print("The total number of Vine reviews with the above criterias are " + str(vine_helpful_df.count()))

The total number of reviews are 904763
The total number of Vine reviews with the above criterias are 14537


In [54]:
# 5-star paid vine reviews 
print("The number of 5-star Paid vine reviews are " 
      + str(vine_paid_df.filter(vine_helpful_df["star_rating"] == "5").count()))

print("The percentage of 5-star Paid vine reviews to Vine reviews is " 
      + str(vine_paid_df.filter(vine_helpful_df["star_rating"] == "5").count() / vine_helpful_df.count() *100) + '%')

print("The percentage of 5-star Paid vine to total number of reviews is " 
      + str(vine_paid_df.filter(vine_helpful_df["star_rating"] == "5").count() / vine_df.count() *100) + '%')

The number of 5-star Paid vine reviews are 34
The percentage of 5-star Paid vine reviews to Vine reviews is 0.23388594620623238%
The percentage of 5-star Paid vine to total number of reviews is 0.0037578901878171415%


In [55]:
# 5-star unpaid vine reviews 
print("The number of 5-star Unpaid vine reviews are " 
      + str(vine_unpaid_df.filter(vine_helpful_df["star_rating"] == "5").count()))

print("The percentage of 5-star Unpaid vine reviews to Vine reviews is " 
      + str(vine_unpaid_df.filter(vine_helpful_df["star_rating"] == "5").count() / vine_helpful_df.count() *100) + '%')

print("The percentage of 5-star Unpaid vine to total number of reviews is " 
      + str(vine_unpaid_df.filter(vine_helpful_df["star_rating"] == "5").count() / vine_df.count() *100) + '%')

The number of 5-star Unpaid vine reviews are 8212
The percentage of 5-star Unpaid vine reviews to Vine reviews is 56.490335007222946%
The percentage of 5-star Unpaid vine to total number of reviews is 0.9076410065398341%
